In [1]:
from google.colab import drive
import os


In [2]:

# Mount Google Drive
drive.mount('/content/drive')

# Path to the main folder (klasifikasi_endek_bali) in Drive
main_folder = '/content/drive/MyDrive/data_endek/'

# Path to the train and test folders
train_folder = main_folder + 'train/'
test_folder = main_folder + 'test/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install -q tensorflow-model-optimization

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_model_optimization as tfmot
import numpy as np


# Load and preprocess your dataset
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Create the base ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Prune the model
end_step = np.ceil(len(train_generator) / 32).astype(np.int32) * 10
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.90, begin_step=0, end_step=end_step)
}
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)



Found 1850 images belonging to 3 classes.
Found 461 images belonging to 3 classes.


In [5]:
from tensorflow.keras.optimizers import Adam  # You can use a different optimizer if needed

# Compile the pruned model
pruned_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:

# Now you can proceed with training
pruned_model.fit(train_generator, validation_data=val_generator, epochs=100)


Epoch 1/100
58/58 [==============================] - 67s 571ms/step - loss: 0.9096 - accuracy: 0.6968 - val_loss: 82.0263 - val_accuracy: 0.3275
Epoch 2/100
58/58 [==============================] - 26s 450ms/step - loss: 0.5569 - accuracy: 0.7400 - val_loss: 1.7678 - val_accuracy: 0.3275
Epoch 3/100
58/58 [==============================] - 26s 452ms/step - loss: 0.5250 - accuracy: 0.7643 - val_loss: 3.7722 - val_accuracy: 0.3275
Epoch 4/100
58/58 [==============================] - 26s 446ms/step - loss: 0.4547 - accuracy: 0.7935 - val_loss: 1.1273 - val_accuracy: 0.3406
Epoch 5/100
58/58 [==============================] - 26s 455ms/step - loss: 0.3888 - accuracy: 0.8346 - val_loss: 8.3551 - val_accuracy: 0.3275
Epoch 6/100
58/58 [==============================] - 26s 445ms/step - loss: 0.3444 - accuracy: 0.8546 - val_loss: 1.1219 - val_accuracy: 0.3731
Epoch 7/100
58/58 [==============================] - 27s 456ms/step - loss: 0.3870 - accuracy: 0.8443 - val_loss: 1.1863 - val_accuracy

In [8]:

# Convert the pruned model to a standard Keras model
final_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Compile the final model
final_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate the final model
test_loss, test_acc = final_model.evaluate(val_generator)
print("Test accuracy:", test_acc)

15/15 [==============================] - 4s 140ms/step - loss: 0.6647 - accuracy: 0.8655
Test accuracy: 0.8655097484588623
